In [ ]:
# -----------------------------------------------------------------------------
# Copyright (c) 2022, Lucid Vision Labs, Inc.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
# EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES
# OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND
# NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS
# BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN
# ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
# CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.
# -----------------------------------------------------------------------------

In [2]:
import ctypes  # ctypes.cast(), ctypes.POINTER(), ctypes.c_ushort
import os  # os.getcwd()
import time
from pathlib import Path

import numpy as np  # pip install numpy
from PIL import Image as PIL_Image  # pip install Pillow

from arena_api.system import system

In [3]:
"""
This function waits for the user to connect a device before raising
an exception
"""
tries = 0
tries_max = 6
sleep_time_secs = 10
while tries < tries_max:  # Wait for device for 60 seconds
    devices = system.create_device()
    if not devices:
        print(
            f'Try {tries+1} of {tries_max}: waiting for {sleep_time_secs} '
            f'secs for a device to be connected!')
        for sec_count in range(sleep_time_secs):
            time.sleep(1)
            print(f'{sec_count + 1 } seconds passed ',
                  '.' * sec_count, end='\r')
        tries += 1
    else:
        print(f'Created {len(devices)} device(s)\n')
        device = devices[0]
        break
else:
    raise Exception(f'No device found! Please connect a device and run '
                    f'the example again.')

Created 1 device(s)



In [4]:
"""
Setup stream nodes
"""
device = devices[0]
print(f'Device used in the example:\n\t{device}')

# Get device stream nodemap
tl_stream_nodemap = device.tl_stream_nodemap

# Enable stream auto negotiate packet size
tl_stream_nodemap['StreamAutoNegotiatePacketSize'].value = True

# Enable stream packet resend
tl_stream_nodemap['StreamPacketResendEnable'].value = True

Device used in the example:
	('1c:0f:af:12:53:65', 'PHX050S-C', 'new', '169.254.3.2')


In [5]:
"""
Configure camera nodes
"""

nodes = device.nodemap.get_node(['Width', 'Height', 'PixelFormat'])

print('Setting Width to its maximum value')
nodes['Width'].value = nodes['Width'].max

print('Setting Height to its maximum value')
height = nodes['Height']
height.value = height.max

# Set pixel format to Mono12, most cameras should support this pixel format
pixel_format_name = 'Mono12'
print(f'Setting Pixel Format to {pixel_format_name}')
nodes['PixelFormat'].value = pixel_format_name

Setting Width to its maximum value
Setting Height to its maximum value
Setting Pixel Format to Mono12


In [6]:
"""
demonstrates Save: Mono12 to PNG
    (1) Setup stream nodes
    (2) Configure camera nodes
    (3) Get buffer data as c pointers
    (4) Convert the data to numpy array using np.ctypeslib
    (5) Convert numpy array to bytes
    (6) Save the byte array as a png
"""

print('Starting stream')
with device.start_stream(1):

    print('Grabbing an image buffer')
    image_buffer = device.get_buffer()

    print(f' Width X Height = '
        f'{image_buffer.width} x {image_buffer.height}')

    """
    To save an image Pillow needs an array that is shaped to
    (height, width). In order to obtain such an array we use numpy
    library
    """
    print('Converting image buffer to a numpy array')

    """
    Buffer.pdata is a (uint8, ctypes.c_ubyte) type
    Buffer.data is a list of elements each represents one byte.
    Since Mono12 uses 16Bits (2 bytes), It is easier to user Buffer.pdata
    over Buffer.data. Buffer.pdata must be cast to (uint16, c_ushort)
    so every element in the array would represent one pixel.
    """
    pdata_as16 = ctypes.cast(image_buffer.pdata,
                            ctypes.POINTER(ctypes.c_ushort))
    nparray_reshaped = np.ctypeslib.as_array(
        pdata_as16,
        (image_buffer.height, image_buffer.width))

    # Saving --------------------------------------------------------------
    print('Saving image')

    png_name = f'from_{pixel_format_name}_to_png_with_pil.png'

    """
    ---------------------------------------------------------------------
    These steps are due to a bug in Pillow saving 16bits png images
    more : https://github.com/python-pillow/Pillow/issues/2970
    """
    nparray_reshaped_as_bytes = nparray_reshaped.tobytes()
    png_array = PIL_Image.new('I', nparray_reshaped.T.shape)
    png_array.frombytes(nparray_reshaped_as_bytes, 'raw', 'I;16')

    png_array.save(png_name)
    print(f'Saved image path is: {Path(os.getcwd()) / png_name }')

    device.requeue_buffer(image_buffer)


Starting stream
Grabbing an image buffer
 Width X Height = 2448 x 2048
Converting image buffer to a numpy array
Saving image
Saved image path is: c:\repos\software\arena_api\examples\from_Mono12_to_png_with_pil.png
Destroyed all created devices


In [ ]:
system.destroy_device()
print('Destroyed all created devices')